# Creation of Municipal Socioeconomic Dataset

Source: Federal Government of Brazil's Cadastro Único (CadÚnico/Cadu)

Author: `Márcio Lopes Jr` 

*Master's student of `Computer Engineering, Intelligent Information Processing` at UFRN-Natal*.

In [ ]:
import pandas as pd
import numpy as np

cat_list = []
val_list = []

**Vartiable Classification Function**

Simple function that attemps to recognise categorical variables. Deciding factors were based on previous knowledge of data, so not a universal function.

In [ ]:
def get_vartypes(df):
    global cat_list, val_list
    cat_list = []
    val_list = []
    for col in df.columns:
        if df[col].nunique() == 2:
            val_list.append(col)
        elif (df[col].nunique() < 21) & \
           (col not in ['qtde_pessoas', 'qtd_comodos_domic_fam', 'qtd_comodos_dormitorio_fam', 
                        'cod_ano_serie_frequenta_memb', 'cod_ano_serie_frequentou_memb', 'qtd_meses_12_meses_memb']):
            cat_list.append(col)
        else:
            val_list.append(col)
            
    return True

**List of Binary Functions**

Exceptional case, categorical columns will be transformed in more the one column (one per category), but in the case of binary variables, only one column will be used, as the categories (2) are complementary anyway.

In [ ]:
binary = ['cod_sabe_ler_escrever_memb', 'cod_deficiencia_memb', 'cod_escola_local_memb', 'cod_concluiu_frequentou_memb', 
          'cod_trabalhou_memb', 'cod_afastado_trab_memb', 'cod_agricultura_trab_memb', 'cod_trabalho_12_meses_memb', 
          'cod_local_domic_fam', 'cod_agua_canalizada_fam', 'cod_banheiro_domic_fam', 'cod_familia_indigena_fam', 
          'ind_familia_quilombola_fam']

**Translations of Codes to Definitions**

In order to rename columns in a more meaningful way. In Education case, translation dictionaries are also used to join columns of past and present education levels, which have different categories.

In [ ]:
# Personal data
freq_to_freq = {1:1, 2:2, 3:3, 4:4, 5:4, 6:5, 7:6, 8:7, 9:8, 10:9, 11:10, 12:11, 14:12, 13:13, 14:14, 15:15}

freq_reduce =  {1 : 'pre', 
                2 : 'pre', 
                3 : 'alfa', 
                4 : 'fund', 
                5 : 'fund', 
                6 : 'fund', 
                7 : 'medio', 
                8 : 'medio', 
                9 : 'eja_fund', 
                10 : 'eja_fund', 
                11 : 'eja_medio', 
                12 : 'alfa_adultos', 
                13 : 'superior', 
                14 : 'pre_vest', 
                15 : 'nenhum'}

trab_translate = {1 : 'autonomo_bico', 
                  2 : 'temp_rural', 
                  3 : 'sem_carteira_ass', 
                  4 : 'com_carteira_ass', 
                  5 : 'dom_sem_carteira_ass', 
                  6 : 'dom_com_carteira_ass', 
                  7 : 'nao_remunerado', 
                  8 : 'militar_ou_servidor', 
                  9 : 'empregador', 
                  10 : 'estagiario',
                  11 : 'aprendiz'}

escola_translate = {1 : 'rede_publica', 
                    2 : 'rede_particular', 
                    3 : 'ja_frequentou', 
                    4 : 'nunca'}

raca_translate = {1 : 'branca', 
                  2 : 'preta', 
                  3 : 'amarela', 
                  4 : 'parda'}

# Household data
piso_translate = {1 : 'terra', 
                  2 : 'cimento', 
                  3 : 'madeira_aproveitada',
                  4 : 'madeira_aparelhada', 
                  5 : 'ceramica_lajota_pedra', 
                  6 : 'carpete'}

mat_translate = {1 : 'alvenaria_tijolo_revestido', 
                 2 : 'alvenaria_tijolo_nao_revestido', 
                 3 : 'madeira_aparelhada', 
                 4 : 'taipa_revestida', 
                 5 : 'taipa_nao_revestida', 
                 6 : 'madeira_aproveitada', 
                 7 : 'palha'}

abaste_translate = {1 : 'rede_geral_dist', 
                    2 : 'poco_nascente', 
                    3 : 'cisterna', 
                    4 : 'outro'}

sanea_translate = {1 : 'rede_esgoto_pluvial', 
                   2 : 'fossa_septica', 
                   3 : 'fossa_rudimentar', 
                   4 : 'vala_ceu_aberto', 
                   5 : 'rio_lago_mar'}

lixo_translate = {1 : 'coletado_diretamento', 
                  2 : 'coletado_indiretamente', 
                  3 : 'queimado_enterrado_propr', 
                  4 : 'terreno_baldio_logradouro', 
                  5 : 'rio_mar'}

iluminacao_translate = {1 : 'eletr_medidor_proprio', 
                        2 : 'eletr_medidor_comunitario', 
                        3 : 'eletr_sem_medidor', 
                        4 : 'oleo_gas_querosene', 
                        5 : 'vela'}

calcamento_translate = {1 : 'total', 
                        2 : 'parcial', 
                        3 : 'nenhum'}

grupo_translate = {101 : 'cigana', 
                   201 : 'extrativista', 
                   202 : 'pesca_artesanal', 
                   203 : 'comunid_terreiro', 
                   204 : 'ribeirinha', 
                   205 : 'agricultura_familiar', 
                   301 : 'assentada_reforma_agraria', 
                   302 : 'benef_prog_nac_cred_fundiario', 
                   303 : 'acampada', 
                   304 : 'atingida_empreend_infra', 
                   305 : 'preso_sist_carcerario', 
                   306 : 'catadores'}

**Personal-level data processing**

Needs to be run first, as only the families of these members will be included in the family calculation.

In [ ]:
df_p = pd.DataFrame()
familias = []
varset = False

for chunk in pd.read_csv("data/base_amostra_pessoa_201812.csv", sep=';', decimal=',', chunksize=500000):
    c = chunk.drop(columns=['estrato', 'classf', 'id_pessoa', 'cod_parentesco_rf_pessoa', 
                            'peso.fam', 'peso.pes', 'cod_local_nascimento_pessoa', 
                            'cod_certidao_registrada_pessoa', 'cod_ano_serie_frequentou_memb'])
    
    print(c.shape)
    c = c[(c.idade > 13) & (c.idade < 41) & (c.cod_sexo_pessoa == 2)]
    familias = familias + c.id_familia.unique().tolist()

    print(c[['val_remuner_emprego_memb', 'val_renda_doacao_memb', 'val_renda_aposent_memb', 'val_renda_seguro_desemp_memb', 'val_renda_pensao_alimen_memb' , 'val_outras_rendas_memb']].sum(axis=1).sum(), c['val_renda_bruta_12_meses_memb'].sum()/13)

    for col in c.columns:
        if col in binary:
            c[col] = c[col].map({2:0, 1:1})

    c['cod_curso_frequentou_pessoa_memb'] = c.cod_curso_frequentou_pessoa_memb.map(freq_to_freq)
    c['cod_curso_memb'] = np.where(c.cod_curso_frequenta_memb.notnull(), c.cod_curso_frequenta_memb, 
                                   c.cod_curso_frequentou_pessoa_memb)
    c['cod_curso_memb'] = c.cod_curso_memb.map(freq_reduce)
    
    c['cod_principal_trab_memb'] = c.cod_principal_trab_memb.map(trab_translate)
    c['ind_frequenta_escola_memb'] = c.ind_frequenta_escola_memb.map(escola_translate)
    c['cod_raca_cor_pessoa'] = c.cod_raca_cor_pessoa.map(raca_translate)
    c.drop(columns=['cod_curso_frequentou_pessoa_memb', 'cod_curso_frequenta_memb', 
                    'cod_ano_serie_frequenta_memb', 'cod_ano_serie_frequentou_memb', 
                    'cod_sexo_pessoa'], inplace=True, errors='ignore')
    
    if varset == False: varset = get_vartypes(c)
        
    c = pd.get_dummies(c, columns=cat_list)

    numero = c.groupby('cd_ibge', as_index=False)[['id_familia']].count().rename(columns={'id_familia':'qnt'})
    c = c.groupby('cd_ibge', as_index=False).mean()
    meancols = list(c.columns)
    c = c.merge(numero, on='cd_ibge')
    c[meancols[2:]] = c[meancols[2:]].multiply(c['qnt'], axis='index')
    df_p = df_p.append(c).fillna(0)
    print(df_p.shape)
    
    
df_pp = df_p.groupby('cd_ibge').sum()
df_pp = df_pp.divide(df_pp.qnt, axis=0).drop(columns=['id_familia', 'qnt'])
print(df_pp.shape)

**Household-level data processing**

In [ ]:
# Base de amostras familiares anonimizadas do CADU 
df = pd.DataFrame()
varset = False
for chunk in pd.read_csv("data/base_amostra_familia_201812.csv", sep=';', decimal=',', chunksize=500000, low_memory=False):
    c = chunk.drop(columns=['estrato', 'classf', 'dat_cadastramento_fam', 'dat_alteracao_fam', 
                            'dat_atualizacao_familia', 'peso.fam', 'cod_centro_assist_fam', 'cod_eas_fam'])
    print(c.shape)
    c = c[(c.id_familia.isin(familias))]

    for col in c.columns:
        if col in binary:
            c[col] = c[col].map({2:0, 1:1})

    c['cod_material_piso_fam'] = c.cod_material_piso_fam.map(piso_translate)
    c['cod_material_domic_fam'] = c.cod_material_domic_fam.map(mat_translate)
    c['cod_abaste_agua_domic_fam'] = c.cod_abaste_agua_domic_fam.map(abaste_translate)
    c['cod_escoa_sanitario_domic_fam'] = c.cod_escoa_sanitario_domic_fam.map(sanea_translate)
    c['cod_destino_lixo_domic_fam'] = c.cod_destino_lixo_domic_fam.map(lixo_translate)
    c['cod_iluminacao_domic_fam'] = c.cod_iluminacao_domic_fam.map(iluminacao_translate)
    c['cod_calcamento_domic_fam'] = c.cod_calcamento_domic_fam.map(calcamento_translate)
    c['ind_parc_mds_fam'] = c.ind_parc_mds_fam.map(grupo_translate)
    
    if varset == False: varset = get_vartypes(c)
        
    c = pd.get_dummies(c, columns=cat_list)
    numero = c.groupby('cd_ibge', as_index=False)[['id_familia']].count().rename(columns={'id_familia':'qnt'})
    c = c.groupby('cd_ibge', as_index=False).mean()
    meancols = list(c.columns)
    c = c.merge(numero, on='cd_ibge')
    c[meancols[2:]] = c[meancols[2:]].multiply(c['qnt'], axis='index')
    
    df = df.append(c).fillna(0)
    print(df.shape)
    

df_pes = df.groupby('cd_ibge').sum()
df_pes = df_pes.divide(df_pes.qnt, axis=0).drop(columns=['id_familia', 'qnt'])

In [ ]:
# Join personal and household data by municipality
df_final = df_pp.join(df_pes).reset_index()

# Add PTB data from the SINASC dataset
ptb_rate_df = pd.read_csv("data/ptb_by_municipality.csv")
df_final = df_final.merge(ptb_rate_df, on='cd_ibge')

print(df_final.shape)

**Special Education Addition**

A municipality with high Middle School education, for instance, will show a small percentage of Primary School education after the application of One-Hot Encoding, but since to study at a certain Education level, one must finish the previous level, the percentage of people at a given level was added to the percentage of people of its previous levels.

In [ ]:
df_final['cod_curso_memb_pre_vest'] += df_final['cod_curso_memb_superior']
df_final['cod_curso_memb_medio'] += df_final['cod_curso_memb_pre_vest'] + df_final['cod_curso_memb_eja_medio']
df_final['cod_curso_memb_fund'] += df_final['cod_curso_memb_medio'] + df_final['cod_curso_memb_eja_fund']
df_final['cod_curso_memb_alfa'] += df_final['cod_curso_memb_fund'] + df_final['cod_curso_memb_alfa_adultos']
df_final['cod_curso_memb_pre'] += df_final['cod_curso_memb_alfa']
df_final['cod_curso_memb_eja'] = df_final['cod_curso_memb_eja_fund'] + df_final['cod_curso_memb_eja_medio'] + df_final['cod_curso_memb_alfa_adultos']

df_final = df_final.drop(columns=['cod_curso_memb_eja_medio', 'cod_curso_memb_eja_fund', 'cod_curso_memb_alfa_adultos'])

**Save file**

In [ ]:
df_final.to_csv("data/pos_means.csv", index=False)